# Experiment Summary (14.08 18:00)

- *Normalization*: Z-score normalization performed per bin
- *Devices*: 18
- *Workers*: 12
- *Participants*: 11
- *Epochs*: 12 epochs for each source
- *Input Vector*: 70 x 1
- *CNN Layers*: 3 layers with 64 filters in the first two layers and 32 filters in the last layer
- *Max Pooling*: Applied twice with a size of 2 x 1
- *Kernel Size*: 5 x 1 x 1
- *Flatten*: Applied after CNN layers before the fully connected layers
- *Fully Connected Layers*: 256, 128, 64, 32, 16, 9
- *Activation Function (before the last layer)*: Tanh
- *Last Layer Type*: Probabilistic with softmax function
- *Loss Method*: Cross-Entropy (CE)
- *Learning Rate*: 0.00001
- *Regularization*: L2 applied
- *Optimizer*: SGD
- *FedServer*: ws 

*Training Phases*:
- *Phase 1*: Participants 1-6
- *Phase 2*: Participants 7-11

*Prediction Phases*: 
- *Phase 1*: Participants 1-6
- *Phase 2*: Participants 7-11


In [4]:
import set_jupyter_env
from apiServer import *
import pandas as pd 

In [5]:
API = ApiServer()

NameError: name 'ApiServer' is not defined

# Experiment Initialization

In [6]:
API.showJsons()

NameError: name 'API' is not defined

In [ ]:
dc = 4
conn = 31
exp = 23
API.setJsons(dc , conn , exp)

In [ ]:
dc_path , conn_path , exp_path = API.getUserJsons()

In [ ]:
exp_name = "EEG_Emotion_18Devices_12Persons_FL"
API.initialization(exp_name, dc_path, conn_path, exp_path)

# Connection Map:

In [ ]:
%pip install networkx pygraphviz
import networkx as nx
def visualize_nerlnet_graph(api_server_inst,  connections : dict , components): # connections is a dictionary with keys as routers and values as lists of their neighbors
    print("Connections: " , list(connections.items()))
    routers = list(connections.keys())
    print("Routers: " , routers)
    workers = list(components.map_worker_to_client.keys())
    print("Workers: " , workers)
    graph = nx.Graph()
    nodes = routers + components.sources + components.clients + workers + [API_SERVER_STR , MAIN_SERVER_STR]
    edges = [] # list of tuples
    for router , neighbors in list(connections.items()):
        for neighbor in neighbors:
            if (router,neighbor) not in edges:
                print(f"Adding edge ({router} , {neighbor}) to graph")
                edges.append((router , neighbor))
    edges.append((API_SERVER_STR , MAIN_SERVER_STR)) # Always connected
    for worker in workers:
        edges.append((worker , components.map_worker_to_client[worker]))
    graph.add_nodes_from(nodes)
    graph.add_edges_from(edges)
    
    my_labels = {'mainServer': 'mS' , 'apiServer': 'aS'}
    nx.relabel_nodes(graph, my_labels , copy=False)
    
    default_colors = {node:'#A90433' for node in graph.nodes()}
    node_colors = {node:default_colors[node] for node in graph.nodes()}
    nx.set_node_attributes(graph, node_colors, 'color')
    colors = nx.get_node_attributes(graph, 'color').values()

    pos = nx.nx_agraph.graphviz_layout(graph)
    angle = 100
    
    plt.figure(figsize=(8,6),dpi=150)
    nx.draw_networkx(graph, pos, with_labels=True, node_color=colors , node_size=200, font_size=8, font_color='white' , edge_color='black' , width=1.5)
    plt.show()
visualize_nerlnet_graph(API , API.json_dir_parser.json_from_path(conn_path)['connectionsMap'] , globe.components)

In [ ]:
API.send_jsons_to_devices()

## Training phase 1:

In [ ]:
API.run_current_experiment_phase()

In [ ]:
stats_train1 = API.get_experiment_flow(exp_name).generate_stats()

In [ ]:
df_loss = stats_train1.get_loss_ts(plot=True, smoothing = True, log_plot = True)

In [ ]:
stats_train1.plot_batches_status(plot=True)


## Training phase 2:

In [ ]:
API.next_experiment_phase()
API.run_current_experiment_phase()

In [ ]:
stats_train2 = API.get_experiment_flow(exp_name).generate_stats()

In [ ]:
stats_train2.get_loss_ts(plot=True, smoothing = True, log_plot = True)

In [ ]:
stats_train2.plot_batches_status(plot=True)

# <span style="color:red;"> *Score Discretization*: </span> Classes are discretized into 3 slots: 1-3, 4-6, 7-9

## Predictaion phase 1:

In [ ]:
API.next_experiment_phase()
API.run_current_experiment_phase()

In [ ]:
stats_pred1 = API.get_experiment_flow(exp_name).generate_stats()

In [ ]:
stats_pred1.plot_batches_status(plot=True)

In [ ]:
conf_source1 , conf_mats1 = stats_pred1.get_confusion_matrices_eeg(plot=True)

In [ ]:
model_performence1 = stats_pred1.get_model_performence_stats(conf_mats1)

In [ ]:
plt.figure(figsize=(12,5))
sns.barplot(data=model_performence.sort_values('Worker') , x='Worker' , y='F1',hue='Class')
plt.title("Worker - F1 Score")
plt.show()
plt.figure(figsize=(12,5))
sns.barplot(data=model_performence.sort_values('Worker') , x='Worker' , y='Accuracy',hue='Class')
plt.title("Worker - Accuracy")
plt.show()
plt.figure(figsize=(12,5))
sns.barplot(data=model_performence.sort_values('Worker') , x='Worker' , y='Precision',hue='Class')
plt.title("Worker - Precision")
plt.show()


## Predictaion phase 2:

In [ ]:
API.next_experiment_phase()
API.run_current_experiment_phase()

In [ ]:
stats_pred2 = API.get_experiment_flow(exp_name).generate_stats()

In [ ]:
stats_pred2.plot_batches_status(plot=True)

In [ ]:
conf_source1 , conf_mats2 = stats_pred2.get_confusion_matrices_eeg(plot=True)

In [ ]:
model_performence2 = stats_pred2.get_model_performence_stats(conf_mats2)